<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Требуется защитить данные клиентов страховой компании. Предлагается разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, а также обосновать корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.

Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

Загрузим данные и сохраним их в переменную df.

In [2]:
df = pd.read_csv('insurance.csv')

Изучим первые 5 строк и основную информацию о таблице, а также проверим наличие дубликатов.

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.duplicated().sum()

153

В данных нет пропусков, все данные имеют корректный тип. В таблице имеется 153 дубликата, однако, т.к. в данных отсутствуют идентификаторы клиентов, мы не можем быть уверены, что это действительно дубликаты, а не просто совпадающие значения, поэтому удалять данные строки не будем.

Также рассмотрим корреляции признаков и целевого признака.

In [6]:
df.corr()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.002074,0.014910,-0.008991,0.010140
Возраст,0.002074,1.000000,-0.019093,-0.006692,0.651030
Зарплата,0.014910,-0.019093,1.000000,-0.030296,-0.014963
Члены семьи,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
Страховые выплаты,0.010140,0.651030,-0.014963,-0.036290,1.000000


Наблюдается средняя линейная зависимость между признаком "Пол" и целевым признаком "Страховые выплаты" (коэффициент корреляции 0.65). Между остальными признаками линейная зависимость отсутствует.

## Умножение матриц

Рассмотрим, как повлияет на качество линейной регрессии умножение матрицы признаков на обратимую матрицу.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

1. Формула получения предсказаний:  
$ a = Xw $  
  
  
2. Формула обучения:  
$ w = (X^T X)^{-1} X^T y $  


3. Объединим обе формулы:  
$ a = X (X^T X)^{-1} X^T y $


4. Выполним умножение матрицы признаков на случайную обратимую матрицу P:  
$ a = X P ((X P)^T X P)^{-1} (X P)^T y$


5. Учитывая свойство матриц $ (AB)^T=B^T A^T $, раскроем скобки:  
$ a = X P (P^T X^T X P)^{-1} P^T X^T y$


6. Так как $ X^T X $ - квадратная матрица, а для квадратных матриц применимо свойство $ (A B C)^{-1} = C^{-1} B^{-1} A^{-1} $, получаем:  
$ a = X P P^{-1} (X^T X) (P^T)^{-1} P^T X^T y$


7. Так как $ P P^{-1} $ и $ P^T (P^T)^{-1} $ это единичные матрицы, умножение на которые не меняет исходную матрицу, сократим уравнение:  
$ a = X (X^T X) X^T y $


Получаем исходную формулу из п.3.

Таким образом, умножение матрицы признаков на обратимую матрицу не влияет на качество линейной регрессии.

## Алгоритм преобразования

Так как умножение матрицы признаков на обратимую матрицу не меняет качество линейной регрессии, используем в качестве алгоритма шифрования умножение матрицы признаков на случайную обратимую матрицу.

Для проверки обратимости матрицы проверим ее ранг (должен быть равен количеству строк), а также наличие обратной для нее матрицы.

Для дешифрования данных необходимо будет умножить зашифрованную матрицу на обратную для случайной матрицу.

Проверим данный алгоритм на практике.

## Проверка алгоритма

Разделим данные на признаки и целевой признак.

In [7]:
features = df.drop(['Страховые выплаты'], axis=1)
target = df['Страховые выплаты']

Разделим признаки и целевой признак на обучающую и тестовую выборки.

In [8]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345)

Обучим линейную регрессию на исходных данных и получим значения метрик MSE, MAE, R2.

In [9]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
mse = mean_squared_error(target_test, predictions)
mae = mean_absolute_error(target_test, predictions)
r2 = r2_score(target_test, predictions)
print('MSE на исходных данных:', mse)
print('MAE на исходных данных:', mae)
print('R2 на исходных данных:', r2)

MSE на исходных данных: 0.11660517472525592
MAE на исходных данных: 0.24631847312116806
R2 на исходных данных: 0.43522757127026546


Теперь создадим случайную матрицу E размера 4x4 (т.к. у нас 4 признака) и обратную к ней матрицу D. В случае, если матрица E окажется необратимой, на экран будет выведено предупреждение.

In [10]:
state = np.random.RandomState(12345)
E = state.randint(1, 10, size=(features.shape[1],features.shape[1]))

In [11]:
try:
    D = np.linalg.inv(E)
except Exception:
    print('Матрица E необратима. Необходимо создать другую матрицу')

Для дополнительной проверки на обратимость проверим ранг матрицы E.

In [12]:
np.linalg.matrix_rank(E)

4

Ранг матрицы совпадает с количеством строк, значим матрица обратима.

Зашифруем признаки на обучающей и тестовой выборках путем умножения на матрицу E.

In [13]:
features_train_enc = features_train @ E
features_test_enc = features_test @ E

Обучим линейную регрессию на зашифрованных данных и получим значения метрик MSE, MAE, R2.

In [14]:
model_enc = LinearRegression()
model_enc.fit(features_train_enc, target_train)
predictions_enc = model_enc.predict(features_test_enc)
mse_enc = mean_squared_error(target_test, predictions_enc)
mae_enc = mean_absolute_error(target_test, predictions_enc)
r2_enc = r2_score(target_test, predictions_enc)
print('MSE на зашифрованных данных:', mse_enc)
print('MAE на зашифрованных данных:', mae_enc)
print('R2 на зашифрованных данных:', r2_enc)

MSE на зашифрованных данных: 0.11660517472524719
MAE на зашифрованных данных: 0.2463184731211494
R2 на зашифрованных данных: 0.43522757127030764


После шифрования данных метрики не изменились.

Также проверим, что после дешифровки качество предсказаний также не поменяется. Расшифруем признаки путем умножения шифрованных данных на матрицу D.

In [15]:
features_train_dec = features_train_enc @ D
features_test_dec = features_test_enc @ D

Обучим линейную регрессию на дешифрованных данных и получим значения метрик MSE, MAE, R2.

In [16]:
model_dec = LinearRegression()
model_dec.fit(features_train_dec, target_train)
predictions_dec = model_dec.predict(features_test_dec)
mse_dec = mean_squared_error(target_test, predictions_dec)
mae_dec = mean_absolute_error(target_test, predictions_dec)
r2_dec = r2_score(target_test, predictions_dec)
print('MSE на дешифрованных данных:', mse_dec)
print('MAE на дешифрованных данных:', mae_dec)
print('R2 на дешифрованных данных:', r2_dec)

MSE на дешифрованных данных: 0.11660517472525739
MAE на дешифрованных данных: 0.2463184731211746
R2 на дешифрованных данных: 0.43522757127025824


**Вывод**

Перед нами стояла задача защитить персональные данные клиентов страховой компании, которые используются для обучения модели машинного обучения.

Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
Целевой признак: количество страховых выплат клиенту за последние 5 лет.

В выборке были представлены данные 5 000 клиентов. В данных отсутствуют пропуски, все данные имеют корректный тип. Также была выявлена средняя линейная зависимость целевого признака ("Страховые выплаты") от одного из признаков ("Пол"), коэффициент корреляции - 0.65.

Теоретически было доказано, что при умножении матрицы признаков на случайную обратимую матрицу качество линейной регрессии не меняется, в связи с чем данный алгоритм был выбран для шифрования данных, а в качестве модели машинного обучения была использована линейная регрессия.

При шифровании данных путем умножения на случайную обратимую матрицу и их последующем дешифровании были получены следующие значения метрик:

MSE на исходных данных: 0.11660517472525592  
MAE на исходных данных: 0.24631847312116806  
R2 на исходных данных: 0.43522757127026546  

MSE на зашифрованных данных: 0.11660517472524719  
MAE на зашифрованных данных: 0.2463184731211494  
R2 на зашифрованных данных: 0.43522757127030764  

MSE на дешифрованных данных: 0.11660517472525739  
MAE на дешифрованных данных: 0.2463184731211746  
R2 на дешифрованных данных: 0.43522757127025824  

Значения метрик практически идентичны на исходных, зашифрованных и дешифрованных данных. Это практически доказывает, что умножение матрицы признаков на обратимую матрицу на влияет на качество линейной регрессии, и данных алгоритм можно использовать для шифрования данных.